In [1]:
import sys
sys.path.append('/home/kvu/erc/libs')

import csv
import os
from sklearn.metrics import confusion_matrix
from tqdm import tqdm

import pandas
import torch
import yaml
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from PIL import Image
from torchvision import transforms as tvtf
from torch.utils.data import DataLoader
from utils.device import detach, move_to

from utils.getter import get_instance
from datasets import *

import warnings
warnings.filterwarnings("ignore")

In [2]:
dataset = ContextAwareDataset(csv_path='/home/kvu/erc/data/raw-audios-wav/train.csv', 
                              audio_feat_dir='/home/kvu/erc/Datasets/MELD/audio-features/train', 
                              text_feat_dir='/home/kvu/erc/Datasets/MELD/text-features/train', 
                              ordered_json_list='/home/kvu/erc/Datasets/MELD/utterance-ordered.json', 
                              num_utt=8)

In [3]:
x, y = dataset[0]

In [4]:
x.shape

AttributeError: 'tuple' object has no attribute 'shape'

In [5]:
y.shape

()

In [6]:
x

(array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.02184726, -0.07112872,  0.04204887, ...,  0.0137796 ,
          0.04377559,  0.01769327],
        [-0.00524412, -0.10400894,  0.05761228, ..., -0.00752982,
          0.02424178, -0.02397549],
        [ 0.00396519, -0.03426354,  0.04282009, ..., -0.01117208,
         -0.008929  ,  0.0490795 ]]),
 array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.02826956, -0.22568929,  0.01267715, ..., -

In [7]:
y

1

In [8]:
dataset_loader = DataLoader(dataset=dataset, batch_size=8, shuffle=False, pin_memory=False) 

In [9]:
x, y = next(iter(dataset_loader))

In [10]:
audio, text = x

In [17]:
audio[0]

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0218, -0.0711,  0.0420,  ...,  0.0138,  0.0438,  0.0177],
        [-0.0052, -0.1040,  0.0576,  ..., -0.0075,  0.0242, -0.0240],
        [ 0.0040, -0.0343,  0.0428,  ..., -0.0112, -0.0089,  0.0491]],
       dtype=torch.float64)

In [13]:
text.shape

torch.Size([8, 8, 1024])